# Automating Presentations with the Databank

In [35]:
#Import All needed Packages
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE
from pptx.dml.color import RGBColor
from pptx.util import Inches, Pt
from pptx.enum.dml import MSO_THEME_COLOR
import numpy as np
import pandas as pd
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.chart.data import ChartData
import numpy as np 
import datetime


from pptx.dml.chtfmt import ChartFormat
from pptx.enum.chart import (
    XL_AXIS_CROSSES,
    XL_CATEGORY_TYPE,
    XL_TICK_LABEL_POSITION,
    XL_TICK_MARK,
)
from pptx.oxml.ns import qn
from pptx.oxml.simpletypes import ST_Orientation
from pptx.shared import ElementProxy
from pptx.text.text import Font, TextFrame
from pptx.util import lazyproperty

In [15]:
#Helper Functions:

#1. Function that creates panel
def create_panel(capital_data):
    #Selecting the initial columns for future processing
    f1 = capital_data.iloc[:, 0:6] 
    #Creating dates out of column names
    Dates_Quarters=capital_data.columns[6:len(capital_data.columns)]
    Dates = pd.to_datetime([f'{x[-4:]}{x[:2]}' for x in Dates_Quarters])
    #Create a Pandas Dataframe with the Dates and size of the new panel
    Dates_pd=pd.DataFrame(Dates)
    Dates_pd.columns=['Dates']
    Dates_column=pd.concat([Dates_pd]*len(f1)).reset_index(drop=True)+ pd.DateOffset(months=2)
    #Create a panel with the first 6 variables, its very imported to sorted by index so I can flip the valus
    Panel=pd.concat([f1]*len(Dates_pd)).sort_index(axis=0)
    Panel=Panel.reset_index(drop=True)
    #Values of the Statistics
    List_of_Rows=[]
    Series=capital_data.iloc[:, 6:]
    for i in range(0,len(Series)):
        list_Series=Series.loc[i].reset_index(drop=True)
        frame=pd.DataFrame(list_Series)
        frame.columns=['Value']
        List_of_Rows.append(frame)
    Values=pd.concat(List_of_Rows)
    Values=Values.reset_index(drop=True)
    # Now we can Concatenate the Dates + Panel + Values
    Result = pd.concat([Dates_column,Panel, Values], axis=1)
    return Result


#2. Separate all the series in the panel and creates a dictionary
def create_series_dictionary(my_panel):
    #Getting Unique Series
    Unique_Series=my_panel.groupby(['Sector','Parameter','Sub-Type','Market','Class']).size().reset_index()
    Unique_Series['Labels']=Unique_Series['Sector'] + "_"+ Unique_Series['Parameter'] + "_" + Unique_Series['Sub-Type'] + "_" + Unique_Series['Market']  + Unique_Series['Class']
    #Dictionary with all the data
    My_Library=dict()
    #Addng the Dates to the Dictionary
    Dates=my_panel['Dates'].dt.strftime("%m/%d/%Y")
    Dates=Dates.unique()   
    My_Library['Dates']=Dates
    #Adding the rest of the series:
    for index,series in enumerate(Unique_Series['Labels']):
        Data=my_panel[(my_panel['Sector'] == Unique_Series['Sector'][index]) & (my_panel['Parameter'] ==  Unique_Series['Parameter'][index])  & (my_panel['Sub-Type'] == Unique_Series['Sub-Type'][index]) & (my_panel['Market'] == Unique_Series['Market'][index]) & (my_panel['Class'] == Unique_Series['Class'][index])]
        Data=Data['Value'].to_numpy()
        My_Library[str(series)]=Data
    return My_Library

#Filling the Array's NAN with value
def fill_array_with_near_value(arr):
    ind = np.where(~np.isnan(arr))[0]
    first, last = ind[0], ind[-1]
    arr[:first] = arr[first]
    arr[last + 1:] = arr[last]
    return arr


In [16]:
#Loading Databank and Creating Panel
capital_data=pd.read_excel(open('Canada CM Databank - Q2 2021_Final_6.xlsx','rb'),sheet_name='Valuation Parameters', header=1,engine = 'openpyxl')

#Creating your Panel, always remeber that there is no empty columns on the
my_panel=create_panel(capital_data)

#Put all the series into a dictionary for use
dictionary_with_panel=create_series_dictionary(my_panel)



In [17]:
#Help Files and Initital Panel
#https://python-pptx.readthedocs.io/en/latest/user/placeholders-using.html
#https://python-pptx.readthedocs.io/en/latest/user/charts.html
my_panel

,Dates,Sector,Market,Sub-Type,Parameter,Class,Source,Value
0,2001-03-01,Industrial,Toronto,Multi Tenant,Average Net Rent,Overall,JLL Research,NaN
1,2001-06-01,Industrial,Toronto,Multi Tenant,Average Net Rent,Overall,JLL Research,NaN
2,2001-09-01,Industrial,Toronto,Multi Tenant,Average Net Rent,Overall,JLL Research,NaN
3,2001-12-01,Industrial,Toronto,Multi Tenant,Average Net Rent,Overall,JLL Research,NaN
4,2002-03-01,Industrial,Toronto,Multi Tenant,Average Net Rent,Overall,JLL Research,NaN
...,...,...,...,...,...,...,...,...
70925,2020-06-01,Industrial,Calgary,Overall,Total Vacancy (s.f.),Overall,NaN,10670651.0
70926,2020-09-01,Industrial,Calgary,Overall,Total Vacancy (s.f.),Overall,NaN,10744288.0
70927,2020-12-01,Industrial,Calgary,Overall,Total Vacancy (s.f.),Overall,NaN,9466855.0
70928,2021-03-01,Industrial,Calgary,Overall,Total Vacancy (s.f.),Overall,NaN,8117932.0


In [19]:
#Variables to Call from Database
dictionary_with_panel.keys()

dict_keys(['Dates', 'Industrial_Average Net Rent_Multi Tenant_CalgaryOverall', 'Industrial_Average Net Rent_Multi Tenant_EdmontonOverall', 'Industrial_Average Net Rent_Multi Tenant_MontrealOverall', 'Industrial_Average Net Rent_Multi Tenant_National AverageOverall', 'Industrial_Average Net Rent_Multi Tenant_Ottawa-GatineauOverall', 'Industrial_Average Net Rent_Multi Tenant_TorontoOverall', 'Industrial_Average Net Rent_Multi Tenant_VancouverOverall', 'Industrial_Average Net Rent_Overall_CalgaryOverall', 'Industrial_Average Net Rent_Overall_EdmontonOverall', 'Industrial_Average Net Rent_Overall_MontrealOverall', 'Industrial_Average Net Rent_Overall_National AverageOverall', 'Industrial_Average Net Rent_Overall_Ottawa-GatineauOverall', 'Industrial_Average Net Rent_Overall_TorontoOverall', 'Industrial_Average Net Rent_Overall_VancouverOverall', 'Industrial_Average Net Rent_Overall_WinnipegOverall', 'Industrial_Average Net Rent_Single Tenant_CalgaryOverall', 'Industrial_Average Net Rent_Sin

# Quarterly Presentation Updates

In [55]:
#Using the Design of JLL Quarterly Market Presentation Report
#1. Open Presentation:
prs=Presentation('Quarterly_Report Slides_Template.pptx')

#2. Slide 1 - Multifamily Cap Rates
lyt=prs.slide_layouts[0]
slide=prs.slides.add_slide(lyt)

#3. Look at placeholders in a slide
for shape in slide.placeholders:
     print('%d %s' % (shape.placeholder_format.idx, shape.name))
        
a,b= slide.placeholders

title=slide.shapes.title
title.text="Canada - Multifamily Cap Rates 2021"


#Create Dates        
from datetime import datetime
#x = [datetime.strptime(s, "%m/%d/%Y") for s in list(dictionary_with_panel['Dates'])]

        
#Create Chart
chart_data = CategoryChartData()
#chart_data.categories = x
chart_data.categories=  list(dictionary_with_panel['Dates'])
chart_data.add_series('Calgary',    fill_array_with_near_value(dictionary_with_panel['Multifamily_Cap Rate_Downtown _CalgaryOverall']))
chart_data.add_series('Edmonton',    fill_array_with_near_value(dictionary_with_panel['Multifamily_Cap Rate_Downtown _EdmontonOverall']))
chart_data.add_series('Vancouver',  fill_array_with_near_value(dictionary_with_panel['Multifamily_Cap Rate_Downtown _VancouverOverall']))
graphic_frame=b.insert_chart(XL_CHART_TYPE.LINE, chart_data)
Actual_graph=graphic_frame.chart

#Title 
#Actual_graph.chart_title.text_frame.text='Response Time in Seconds'
#Actual_graph.chart_title.has_text_frame=False
#Actual_graph.chart_title.text_frame.paragraphs[0].font.size = Pt(16)

#Legend
#Actual_graph.legend.include_in_layout=True

#Global Font Size
Actual_graph.font.size = Pt(11)
Actual_graph.font.name = "Source Sans Pro"

#Y Axix
Actual_graph.value_axis.axis_title.text_frame.add_paragraph="Cap Rate (%)"
Actual_graph.value_axis.has_major_gridlines=False

#X Axisx , Import format the text asis, and put the tick marks on tick axisx position.


#Save presentation
prs.save("Testing8.pptx")



0 Title 1
10 Chart Placeholder 2


In [50]:
def set_on_tick_marks(axis):
    """
    Workaround function that implements the on tick marks of the axis position
    :param axis: The axis element of a chart
    """
    cross_between_elements = axis._element.xpath(r"c:crossBetween")
    if cross_between_elements:
        cross_between_element = cross_between_elements[0]
    else:
        cross_between_element = OxmlElement(r"c:crossBetween")
        axis._element.append(cross_between_element)
    cross_between_element.set("val", "midCat")

# Investment Outlook

In [42]:
#Using the Design of JLL Investment Outlook
#1. Open Presentation:
prs=Presentation('Templates_Slide_Master.pptx')
# Slide # 1
#There are 0-9 slide layouts from Slide Master
lyt=prs.slide_layouts[3] # choosing a slide layout
slide=prs.slides.add_slide(lyt) # adding a slide
#title=slide.shapes.title # assigning a title
#title.text="Testing the Second type!" # title

#2. Look at placeholders in a slide
for shape in slide.placeholders:
     print('%d %s' % (shape.placeholder_format.idx, shape.name))

#This is Very important! Look at all the place holders in the powerpoint slide master, assing them
a, b, c,d,e,f= slide.placeholders

a.text="hello my name is reinaldo"
b.text="Box for Industrial text "
c.text=" Box of the Land and Development box"

#Testing the Chart1
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.chart.data import ChartData
import numpy as np 
import datetime
N = 100
random_x = np.random.randn(N) + 10
random_y = np.random.randn(N)+5
random_z = np.random.randn(N) +20
dte=datetime.datetime.today()
dt_lst=[dte-datetime.timedelta(days=i) for i in range(N)]
chart_data = ChartData()
chart_data.categories = dt_lst
chart_data.add_series('Data 1',    random_x)
chart_data.add_series('Data 2',    random_y)
chart_data.add_series('Data 3',    random_z)
graphic_frame=d.insert_chart(XL_CHART_TYPE.LINE, chart_data)
Actual_graph=graphic_frame.chart

Actual_graph.has_legend = True

#Global Font Size
Actual_graph.font.size = Pt(8)

#Font Size title
Actual_graph.chart_title.text_frame.text='Response Time in Seconds'
Actual_graph.chart_title.text_frame.paragraphs[0].font.size = Pt(10)
Actual_graph.legend.include_in_layout


prs.save("Testing6.pptx") # saving file

1 Content Placeholder 6
10 Text Placeholder 1
11 Text Placeholder 2
13 Chart Placeholder 3
14 Chart Placeholder 4
15 Chart Placeholder 5


In [184]:
random_x
#np.nan_to_num(dictionary_with_panel['Multifamily_Cap Rate_Downtown _CalgaryOverall'])

array([10.47083125,  7.89415333,  9.79725066,  9.05318972,  9.68397362,
       11.45216951,  9.50345343,  9.28458531,  8.54805216, 10.80067677,
       10.98875648, 10.75497324, 10.98310823, 12.46654094,  9.977391  ,
       10.74331879,  9.53840477,  8.30320228, 11.74455639, 10.13843354,
        9.2343374 ,  9.29733098,  9.85731138, 11.09543712,  8.87590321,
        9.39220551,  9.24992257,  9.78851764,  9.21415492, 10.02361898,
       10.8396559 ,  9.46928075,  7.15796393,  8.01812689,  8.99704124,
       12.28552623,  8.8248007 ,  9.36391937, 10.88050972, 10.99805312,
       10.51355465,  9.78297165, 10.00830667, 10.44131678, 12.18984633,
        9.67207314,  9.43065717, 10.27098133, 10.37445789,  9.94839729,
       11.59292411,  9.28352948,  9.45570027,  8.70292277,  9.83547654,
        9.15160456, 10.99273391, 10.5296076 , 10.89665913,  9.33589465,
       10.49936797,  9.72947227,  9.38121151,  9.26844701,  9.41071163,
        9.81287401,  9.99891268,  7.99954009, 11.46962395,  9.29

In [ ]:
from datetime import datetime

datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

In [91]:
prs = Presentation()
slide = prs.slides.add_slide(prs.slide_layouts[6])
chart_data = ChartData()
chart_data.categories = ['East', 'West', 'Midwest']
chart_data.add_series('Q1 Sales', (19.2, 21.4, 16.7))
chart_data.add_series('Q2 Sales', (22.3, 28.6, 15.2))
chart_data.add_series('Q3 Sales', (20.4, 26.3, 14.2))

x, y, cx, cy = Inches(2), Inches(2), Inches(6), Inches(4.5)
columnc = slide.shapes.add_chart(
    XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data
)

prs1 = Presentation()
slide1 = prs1.slides.add_slide(prs1.slide_layouts[6])
chart_data1 = ChartData()
chart_data1.categories = ['East', 'West', 'Midwest']
chart_data1.add_series('Q1 Sales', (5, 5, 5))

linec = slide1.shapes.add_chart(
    XL_CHART_TYPE.LINE, x, y, cx, cy, chart_data1
)
chartb = columnc.chart
chartl = linec.chart

a=chartl.plots._plotArea
b=chartb.plots._plotArea
#print a.xml
#dir(a)
k=a.getchildren()[0]
#print b.xml
b.insert(1,k)
#print b.xml
prs.save('test.pptx')

In [204]:
Dates=my_panel['Dates'].dt.strftime("%m/%d/%Y").to_numpy()

In [212]:
Dates=my_panel['Dates'].dt.strftime("%m/%d/%Y")
Dates.unique()

array(['01/01/2001', '04/01/2001', '07/01/2001', '10/01/2001',
       '01/01/2002', '04/01/2002', '07/01/2002', '10/01/2002',
       '01/01/2003', '04/01/2003', '07/01/2003', '10/01/2003',
       '01/01/2004', '04/01/2004', '07/01/2004', '10/01/2004',
       '01/01/2005', '04/01/2005', '07/01/2005', '10/01/2005',
       '01/01/2006', '04/01/2006', '07/01/2006', '10/01/2006',
       '01/01/2007', '04/01/2007', '07/01/2007', '10/01/2007',
       '01/01/2008', '04/01/2008', '07/01/2008', '10/01/2008',
       '01/01/2009', '04/01/2009', '07/01/2009', '10/01/2009',
       '01/01/2010', '04/01/2010', '07/01/2010', '10/01/2010',
       '01/01/2011', '04/01/2011', '07/01/2011', '10/01/2011',
       '01/01/2012', '04/01/2012', '07/01/2012', '10/01/2012',
       '01/01/2013', '04/01/2013', '07/01/2013', '10/01/2013',
       '01/01/2014', '04/01/2014', '07/01/2014', '10/01/2014',
       '01/01/2015', '04/01/2015', '07/01/2015', '10/01/2015',
       '01/01/2016', '04/01/2016', '07/01/2016', '10/01

In [14]:
from datetime import datetime
from dateutil.relativedelta import relativedelta


capital_data=pd.read_excel(open('Canada CM Databank - Q2 2021_Final_6.xlsx','rb'),sheet_name='Valuation Parameters', header=1,engine = 'openpyxl')
#Selecting the initial columns for future processing
f1 = capital_data.iloc[:, 0:6] 
#Creating dates out of column names
Dates_Quarters=capital_data.columns[6:len(capital_data.columns)]
Dates = pd.to_datetime([f'{x[-4:]}{x[:2]}' for x in Dates_Quarters])
Dates_pd=pd.DataFrame(Dates)
Dates_pd.columns=['Dates']
Dates_column=pd.concat([Dates_pd]*len(f1)).reset_index(drop=True)


Dates_column + pd.DateOffset(months=2)

,Dates
0,2001-03-01
1,2001-06-01
2,2001-09-01
3,2001-12-01
4,2002-03-01
...,...
70925,2020-06-01
70926,2020-09-01
70927,2020-12-01
70928,2021-03-01


In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
given_date = '21/1/2021'
print('Give Date: ', given_date)
date_format = '%d/%m/%Y'
dtObj = datetime.strptime(given_date, date_format)
# Add 20 months to a given datetime object
n = 20
future_date = dtObj + relativedelta(months=n)
print('Date after 20 months: ', future_date)
print('Date after 20 months: ', future_date.date())
# Convert datetime object to string in required format
future_date_str = future_date.strftime(date_format)
print('Date after 20 months (as string): ', future_date_str)